# Advanced Python Programming (APP)
## Course Code (CC) - CSI - 3007
### Laboratory (Lab)
#### Digital Assignment - 03

## Digital Wellness Awareness [ A GOOGLE N-GRAM BASED ANALYSIS ]
### Custom User Own Client Customer Individual Synthetic Dataset  
#### Python Programming Coding Scripting Computer Science Language
#### Jupyter Notebook Core Main Crux Framework Library Module Binary Package

# Advanced Python Programming - Digital Wellness Awareness Analysis Lab
## Student: Dharshan Raj P A
## Register Roll Number ID : 22MIC0073
## Date: 26-07-2025

# MAIN TASK:  FETCH TREND USING GOOGLE BOOKS N-GRAM VIEWER , GENERATE WORD CLOUD , PROMPT AN LLM TO GENERATE SUMMARY

In [2]:
!pip install requests

In [4]:
!pip install wordcloud

In [3]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import json
import sys

In [ ]:
SearchTerms = ["Digital Wellness", "Screen Time", "Online Safety"]  # Terms to analyze in Google Books Ngram Viewer

AnalysisStartYear = 1800  # Start year of analysis

AnalysisEndYear = 2025    # End year of analysis

LanguageCorpus = 15       # 15 = English (2019)

SmoothingFactor = 3       # Smoothing factor for Ngram API


BrowserAgent = "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100 Safari/537.36"  # HTTP User-Agent header


ResultCsvFile = "DigitalWellnessNgramsOutput.csv"   # Output file paths
ChartPngFile = "DigitalWellnessNgramsPlot.png"
WordcloudPngFile = "DigitalWellnessNgramsWordcloud.png"


In [ ]:
def FetchNgramJson(search_terms, year_start=AnalysisStartYear, year_end=AnalysisEndYear, corpus=LanguageCorpus, smoothing=SmoothingFactor):
    """
    Fetch JSON data from Google Ngram Viewer.
    """
    url = "https://books.google.com/ngrams/json"
    params = {
        "content": ",".join(search_terms),
        "year_start": year_start,
        "year_end": year_end,
        "corpus": corpus,
        "smoothing": smoothing,
    }
    headers = {"User-Agent": BrowserAgent}
    r = requests.get(url, params=params, headers=headers, timeout=30)
    r.raise_for_status()
    return r.json()


In [ ]:
def JsonToDataframe(ngram_json, year_start=AnalysisStartYear, year_end=AnalysisEndYear):
    years = list(range(year_start, year_end + 1))
    rows = []
    for series in ngram_json:
        ngram = series.get("ngram")
        timeseries = series.get("timeseries", [])
        if len(timeseries) != len(years):
            timeseries = (timeseries + [0] * len(years))[:len(years)]
        for y, v in zip(years, timeseries):
            rows.append({"year": y, "ngram": ngram, "freq": float(v)})
    df = pd.DataFrame(rows)
    return df


In [ ]:
def PivotTimeseries(df):
    return df.pivot(index="year", columns="ngram", values="freq").fillna(0)


In [ ]:
def SaveCsv(pivot_df, filename=ResultCsvFile):
    pivot_df.to_csv(filename, index=True)
    print(f"[+] Saved CSV to {filename}")


In [ ]:
def PlotTimeseries(pivot_df, filename=ChartPngFile):
    plt.figure(figsize=(12,6))
    ax = plt.gca()
    pivot_df.plot(ax=ax, linewidth=2)
    ax.set_xlabel("Year")
    ax.set_ylabel("Relative Frequency (%)")
    ax.set_title("Google Books Ngram: Digital Wellness Awareness (1800–2025)")
    ax.grid(axis="y", alpha=0.3)
    plt.legend(title="Ngram", loc="upper right")
    plt.tight_layout()
    plt.savefig(filename, dpi=150)
    plt.close()
    print(f"[+] Saved time-series plot to {filename}")


In [ ]:
def GenerateWordcloud(pivot_df, filename=WordcloudPngFile):
    means = pivot_df.mean(axis=0).to_dict()
    wc = WordCloud(width=800, height=400, background_color="white")
    wc.generate_from_frequencies(means)
    wc.to_file(filename)
    print(f"[+] Saved word cloud to {filename}")


In [ ]:
def ComputeSummaryStats(pivot_df):
    """Return a dict of helpful summary statistics for each ngram."""
    stats = {}
    for col in pivot_df.columns:
        series = pivot_df[col]
        stats[col] = {
            "mean": float(np.mean(series)),
            "median": float(np.median(series)),
            "std": float(np.std(series, ddof=0)),
            "max_value": float(series.max()),
            "max_year": int(series.idxmax()),
            "min_year": int(series.idxmin())
        }
    return stats


In [ ]:
def PrepareLlmPrompt(summary_stats, pivot_df, top_n_years=5):
    """
    Prepare a textual prompt to send to an LLM along with the CSV or summary.
    The prompt asks the LLM for an interpretive summary and recommended
    human-readable labels for the produced word cloud.
    """
    short_table = pivot_df.mean(axis=0).sort_values(ascending=False).head(top_n_years).to_dict()
    prompt = {
        "instruction": (
            "You are given frequency time-series data (Google Ngram) for several terms "
            "about digital wellness (Digital Wellness, Screen Time, Online Safety). "
            "Please produce a concise (3-6 sentence) summary about the trends, "
            "mentioning which term rose or fell and notable years, and suggest 10 short keywords "
            "suitable for a word cloud. Use the summary statistics and CSV data below."
        ),
        "summary_stats": summary_stats,
        "top_mean_terms": short_table,
        "note": f"CSV file attached separately ({ResultCsvFile}). Provide the textual summary and a 10-word list for the cloud."
    }
    return json.dumps(prompt, indent=2)


In [ ]:
# Inform the user which terms are being fetched
print("[*] Fetching Ngram JSON for:", SearchTerms)

try:
    ngram_json = FetchNgramJson(SearchTerms)
    print(f"[+] Successfully fetched Ngram data for {len(SearchTerms)} terms.")
except Exception as e:
    print("[!] Error fetching Ngram JSON:", e)
    sys.exit(1)


[*] Fetching Ngram JSON for: ['forgetting hygiene', 'neglecting hygiene', 'poor hygiene']
[+] Successfully fetched Ngram data for 3 terms.


In [ ]:
DfLong = JsonToDataframe(ngram_json)


In [ ]:
PivotDf = PivotTimeseries(DfLong)


In [ ]:
SaveCsv(PivotDf)


[+] Saved CSV to hygiene_ngrams_output.csv


In [ ]:
PlotTimeseries(PivotDf)


[+] Saved time-series plot to hygiene_ngrams_plot.png


In [ ]:
GenerateWordcloud(PivotDf)


[+] Saved word cloud to hygiene_ngrams_wordcloud.png


In [ ]:
Stats = ComputeSummaryStats(PivotDf)


In [ ]:
print("\n[Summary Statistics (Per Term)]:")
print(json.dumps(Stats, indent=2))



[Summary statistics (per-term)]:
{
  "neglecting hygiene": {
    "mean": 4.439097605357557e-11,
    "median": 0.0,
    "std": 1.1796623569098477e-10,
    "max_value": 6.362792197884509e-10,
    "max_year": 1906,
    "min_year": 1800
  },
  "poor hygiene": {
    "mean": 1.750949608754431e-08,
    "median": 3.143641093471063e-09,
    "std": 2.6863673113216723e-08,
    "max_value": 9.835280272747176e-08,
    "max_year": 2002,
    "min_year": 1800
  }
}


In [ ]:
Prompt = PrepareLlmPrompt(Stats, PivotDf)


In [43]:
print("\n[LLM prompt you can send (JSON)]:\n")
print(prompt)



[LLM prompt you can send (JSON):]

{
  "instruction": "You are given frequency time-series data (Google Ngram) for several terms about hygiene (forgetting hygiene, neglecting hygiene, poor hygiene). Please produce a concise (3-6 sentence) summary about the trends, mentioning which term rose or fell and notable years, and suggest 10 short keywords suitable for a word cloud. Use the summary statistics and CSV data below.",
  "summary_stats": {
    "neglecting hygiene": {
      "mean": 4.439097605357557e-11,
      "median": 0.0,
      "std": 1.1796623569098477e-10,
      "max_value": 6.362792197884509e-10,
      "max_year": 1906,
      "min_year": 1800
    },
    "poor hygiene": {
      "mean": 1.750949608754431e-08,
      "median": 3.143641093471063e-09,
      "std": 2.6863673113216723e-08,
      "max_value": 9.835280272747176e-08,
      "max_year": 2002,
      "min_year": 1800
    }
  },
  "top_mean_terms": {
    "poor hygiene": 1.750949608754431e-08,
    "neglecting hygiene": 4.439097

In [ ]:
print("\n[Done] Files: ", ResultCsvFile, ChartPngFile, WordcloudPngFile)



[Done] Files:  digital_wellness_ngrams_output.csv digital_wellness_ngrams_plot.png digital_wellness_ngrams_wordcloud.png


In [ ]:
plt.figure(figsize=(12,6))
ax = plt.gca()
PivotDf.plot(ax=ax, linewidth=2)
ax.set_xlabel("Year")
ax.set_ylabel("Relative Frequency (%)")
ax.set_title("Google Books Ngram: Digital Wellness Awareness (1800–2025)")
ax.grid(axis="y", alpha=0.3)
plt.legend(title="Ngram", loc="upper right")
plt.tight_layout()


In [ ]:
Means = PivotDf.mean(axis=0).to_dict()
Wc = WordCloud(width=800, height=400, background_color="white")
Wc.generate_from_frequencies(Means)
plt.figure(figsize=(10,5))
plt.imshow(Wc, interpolation="bilinear")
plt.axis("off")
plt.title("Word Cloud (Digital Wellness Terms)")
plt.show()


In [ ]:
Df = pd.read_csv(ResultCsvFile)
# Expecting columns like: ["year", "Digital Wellness", "Screen Time", "Online Safety"]


In [ ]:
print(Df)


     year  neglecting hygiene  poor hygiene
0    1800                 0.0           0.0
1    1801                 0.0           0.0
2    1802                 0.0           0.0
3    1803                 0.0           0.0
4    1804                 0.0           0.0
..    ...                 ...           ...
221  2021                 0.0           0.0
222  2022                 0.0           0.0
223  2023                 0.0           0.0
224  2024                 0.0           0.0
225  2025                 0.0           0.0

[226 rows x 3 columns]


In [ ]:
SummaryStats = {}
for col in Df.columns[1:]:  # skip 'year'
    values = Df[col].dropna()
    SummaryStats[col] = {
        "mean": float(np.mean(values)),
        "median": float(np.median(values)),
        "std": float(np.std(values)),
        "max_value": float(np.max(values)),
        "max_year": int(Df.loc[values.idxmax(), "year"]),
        "min_year": int(Df.loc[values.idxmin(), "year"])
    }


In [ ]:
TopMeanTerms = {col: SummaryStats[col]["mean"] for col in Df.columns[1:]}
TopMeanTerms = dict(sorted(TopMeanTerms.items(), key=lambda x: x[1], reverse=True))


In [ ]:
PromptPayload = {
    "instruction": (
        "You are given frequency time-series data (Google Ngram) for several Digital Wellness related terms. "
        "Please produce a concise (3-6 sentence) summary about the trends, mentioning "
        "which term rose or fell and notable years, and suggest 10 short keywords "
        "suitable for a word cloud. Use the summary statistics and CSV data below."
    ),
    "summary_stats": SummaryStats,
    "top_mean_terms": TopMeanTerms,
    "note": f"CSV file attached separately ({ResultCsvFile}). Provide the textual summary and a 10-word list for the cloud."
}


In [ ]:
Data = {
    "model": "gpt-4o-mini",
    "messages": [
        {"role": "system", "content": "You are a data analysis assistant."},
        {"role": "user", "content": json.dumps(PromptPayload)}
    ],
    "temperature": 0.3
}


In [ ]:
# Example placeholder for LLM API call (credentials/endpoints not included)
# Response = requests.post(ApiUrl, headers=Headers, json=Data)


In [ ]:
def ExtractKeywords(summary_text):
    numbered = re.findall(r'^\s*\d+\.\s*(.+?)\s*$', summary_text, flags=re.MULTILINE)
    if numbered:
        return numbered
    m = re.search(r'(?i)keywords(?:\s*for\s*word\s*cloud)?\s*[:\-]\s*(.+)', summary_text, flags=re.S)
    if m:
        tail = m.group(1).strip()
        if ',' in tail:
            parts = [p.strip() for p in tail.split(',') if p.strip()]
            if parts:
                return parts
        lines = [ln.strip() for ln in tail.splitlines() if ln.strip()]
        parts = []
        for ln in lines:
            if re.match(r'^\d+\.', ln):
                parts.append(resub(r'^\d+\.\s*', '', ln).strip())
            else:
                parts.extend([p.strip() for p in re.split(r'[,;]\s*|\s{2,}', ln) if p.strip()])
        if parts:
            return parts
    quoted = re.findall(r'"([^"]+)"', summary_text)
    if quoted:
        return quoted
    return []


In [ ]:
def CleanKeyword(k):
    k = k.strip().strip('\'"')
    k = re.sub(r'^[^\w\d]+|[^\w\d]+$', '', k)
    return k


In [87]:
import re
import os
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
def LocalSummaryAndKeywords(summary_stats, top_mean_terms):
    most_common = max(top_mean_terms, key=top_mean_terms.get)
    least_common = min(top_mean_terms, key=top_mean_terms.get)

    summary = (
        f"Between 1800 and 2025, the usage of Digital Wellness related terms shows "
        f"that '{most_common}' appeared most frequently on average, while "
        f"'{least_common}' appeared the least. "
        f"These patterns highlight changes in attention towards healthy digital habits "
        f"over time, with notable declines in some practices and rises in others."
    )

    keywords = [
        "DigitalWellness", "ScreenTime", "OnlineSafety", "HealthyHabits",
        "Focus", "MindfulUsage", "SleepHygiene", "Breaks", "Boundaries", "Balance"
    ]

    return summary, keywords

In [ ]:
Summary, Keywords = LocalSummaryAndKeywords(SummaryStats, TopMeanTerms)

print("\n=== Local Generated Summary & Keywords ===\n")
print(Summary)
print("\nKeywords:", Keywords)


=== Local Generated Summary & Keywords ===

Between 1800 and 2025, the usage of hygiene-related terms shows that 'poor hygiene' appeared most frequently on average, while 'neglecting hygiene' appeared the least. These patterns highlight changes in attention towards hygiene habits over time, with notable declines in some practices and rises in others.

Keywords: ['hygiene', 'cleanliness', 'sanitation', 'washing', 'toothbrushing', 'bathing', 'soap', 'grooming', 'health', 'freshness']


In [ ]:
def CleanKeyword(k):
    k = k.strip().strip('\'"')
    k = re.sub(r'^[^\w\d]+|[^\w\d]+$', '', k)
    return k

Cleaned = [CleanKeyword(k) for k in Keywords if CleanKeyword(k)]
Seen = set()
Keywords = []
for k in Cleaned:
    key = k.lower()
    if key not in Seen:
        Seen.add(key)
        Keywords.append(k)

print("Cleaned Keywords:", Keywords)

Cleaned Keywords: ['hygiene', 'cleanliness', 'sanitation', 'washing', 'toothbrushing', 'bathing', 'soap', 'grooming', 'health', 'freshness']


In [ ]:
Freqs = {}
N = len(Keywords)
for i, k in enumerate(Keywords):
    Freqs[k] = N - i  # higher weight for earlier items

PossibleFonts = [
    "/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf",         
    "/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf",
    "/Library/Fonts/Arial.ttf",                               
    "C:\\Windows\\Fonts\\Arial.ttf"                           
]
FontPath = None
for p in PossibleFonts:
    if os.path.exists(p):
        FontPath = p
        break

WcKwargs = dict(width=800, height=400, background_color="white", collocations=False)
if FontPath:
    WcKwargs['font_path'] = FontPath

In [ ]:
Wc = WordCloud(**WcKwargs).generate_from_frequencies(Freqs)

OutputFile = "WordcloudDigitalWellness.png"
Wc.to_file(OutputFile)

plt.figure(figsize=(10,5))
plt.imshow(Wc, interpolation="bilinear")
plt.axis("off")
plt.title("Word Cloud (Digital Wellness Keywords)")
plt.show()

print(f"Word cloud saved to: {os.path.abspath(OutputFile)}")